# Imports

In [1]:
from utils.nfl_data import (get_weekly_df, get_seasonal_df, get_roster_df,
                            create_mappings, create_position_groups, create_positional_df)

# Setting Years List

The years past into this list provides us with the data in each of those seasons.

In [2]:
year = 2023
years = [year]

# Seasonal and Weekly Data

In [3]:
df_weekly = get_weekly_df(years=years, columns=[])

Downcasting floats.


In [4]:
df_seasonal = get_seasonal_df(years=years)

# Roster ID

For player mapping creations

In [5]:
columns = ["player_name", "player_id", "position"]
df_roster = get_roster_df(years=years, columns=columns)

# Create Player Mappings

In [6]:
player_map, id_map = create_mappings(df_roster=df_roster, df_seasonal=df_seasonal, df_weekly=df_weekly)

## Mapping Checks

In [7]:
# kelce_id = id_map["Travis Kelce"]

In [8]:
# player_map[kelce_id]["seasonal_df"]

In [9]:
# player_map[kelce_id]["week_dfs"][-1]

# Creating Position Groups

In [10]:
qbs, rbs, wrs, tes = create_position_groups(player_map=player_map)

In [11]:
df_qb = create_positional_df(qbs)
df_rb = create_positional_df(rbs)
df_wr = create_positional_df(wrs)
df_te = create_positional_df(tes)

In [12]:
df_qb.head(1)

,Index,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh
0,0,00-0019596,2021,REG,485,719,5316.0,43,12.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.195705


In [13]:
df_rb.head(1)

,Index,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh
0,7,00-0025394,2021,REG,0,0,0.0,0,0.0,0.0,...,0.031746,0.036857,0.008791,0.0,0.0,0.0,0.004396,0.007033,0.055944,0.078979


In [14]:
df_wr.head(1)

,Index,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh
0,8,00-0026035,2021,REG,0,1,0.0,0,0.0,0.0,...,0.134443,0.343891,0.140192,0.272727,0.131579,0.149425,0.20646,0.166699,0.953846,0.111496


In [15]:
df_te.head(1)

,Index,player_id,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,yac_sh,wopr_y,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,dom,w8dom,yptmpa,ppr_sh
0,5,00-0024243,2021,REG,0,0,0.0,0,0.0,0.0,...,0.07856,0.11565,0.064497,0.0,0.057803,0.05,0.032248,0.051597,0.467249,0.036543


# Saving Data to CSV for Graphing Tools

In [16]:
def clean_df_for_csv(df):
    """"""
    df["name"] = df["player_id"].map(lambda x: player_map[x]['name'])
    df.drop(columns=["player_id", "Index"], inplace=True)
    
    df["total_epa"] = df["passing_epa"] + df["rushing_epa"]
    
    ignore_cols = ["name", "season", "season_type", "games", "target_share", "air_yards_share"
                  "tgt_sh", "ay_sh", "yac_sh", "ry_sh", "rtd_sh", "rfd_sh", "rtdfd_sh", "ppr_sh"]
    for col in df_qb.columns.to_list():
        if col not in ignore_cols:
            df[f"{col} per game"] = df[col] / df["games"]
            
    df = df[['name'] + [col for col in df.columns if col != "name"]]
    return df

## Quarterback Save

In [17]:
df_to_save_qb = clean_df_for_csv(df_qb)
df_to_save_qb.head(1)

,name,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,...,wopr_x per game,special_teams_tds per game,fantasy_points per game,fantasy_points_ppr per game,tgt_sh per game,wopr_y per game,dom per game,w8dom per game,yptmpa per game,total_epa per game
0,Tom Brady,2021,REG,485,719,5316.0,43,12.0,22.0,144.0,...,0.0,0.0,22.043529,22.043529,0.0,0.0,0.0,0.0,0.0,8.797962


In [18]:
df_to_save_qb.to_csv(f"FantasyData/advanced-stats/QB/{year}.csv")

## Running Back Save

In [19]:
df_to_save_rb = clean_df_for_csv(df_rb)
df_to_save_rb.head(1)

,name,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,...,wopr_x per game per game,special_teams_tds per game per game,fantasy_points per game per game,fantasy_points_ppr per game per game,tgt_sh per game per game,wopr_y per game per game,dom per game per game,w8dom per game per game,yptmpa per game per game,total_epa per game per game
0,Adrian Peterson,2021,REG,0,0,0.0,0,0.0,0.0,0.0,...,0.008538,0.0,1.4125,1.6625,0.001748,0.002304,0.000275,0.00044,0.003497,-0.616539


In [20]:
df_to_save_rb.to_csv(f"FantasyData/advanced-stats/RB/{year}.csv")

# Wide Receiver Save

In [21]:
df_to_save_wr = clean_df_for_csv(df_wr)
df_to_save_wr.head(1)

,name,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,...,wopr_x per game per game,special_teams_tds per game per game,fantasy_points per game per game,fantasy_points_ppr per game per game,tgt_sh per game per game,wopr_y per game per game,dom per game per game,w8dom per game per game,yptmpa per game per game,total_epa per game per game
0,Danny Amendola,2021,REG,0,1,0.0,0,0.0,0.0,0.0,...,0.040545,0.0,0.7,1.075,0.002284,0.005373,0.003226,0.002605,0.014904,-0.005652


In [22]:
df_to_save_wr.to_csv(f"FantasyData/advanced-stats/WR/{year}.csv")

# Tight End Save

In [23]:
df_to_save_te = clean_df_for_csv(df_te)
df_to_save_te.head(1)

,name,season,season_type,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,...,wopr_x per game per game,special_teams_tds per game per game,fantasy_points per game per game,fantasy_points_ppr per game per game,tgt_sh per game per game,wopr_y per game per game,dom per game per game,w8dom per game per game,yptmpa per game per game,total_epa per game per game
0,Marcedes Lewis,2021,REG,0,0,0.0,0,0.0,0.0,0.0,...,0.009174,0.0,0.126627,0.262722,0.000362,0.000684,0.000191,0.000305,0.002765,0.0


In [24]:
df_to_save_te.to_csv(f"FantasyData/advanced-stats/TE/{year}.csv")